In [1]:
import numpy as np
from NonlinearController.mpc_utils import *
from NonlinearController.controllers import *
from NonlinearController.model_utils import *
from NonlinearController.lpv_embedding import *
from NonlinearController.systems import UnbalancedDisc
import deepSI

In [2]:
model = deepSI.load_system("NonlinearController/trained_models/unbalanced/ObserverUnbalancedDisk_dt01_nab_4_SNR_30_e250")

nx = model.nx
nu = model.nu if model.nu is not None else 1
ny = model.ny if model.ny is not None else 1

In [3]:
class velocity_lpv_embedder():
    def __init__(self, ss_enc, Nc, n_stages=20):
        self.nx = ss_enc.nx
        self.nu = ss_enc.nu if ss_enc.nu is not None else 1
        self.ny = ss_enc.ny if ss_enc.ny is not None else 1

        self.Nc = Nc

        self.lpv_A, self.lpv_B, self.lpv_C = velocity_lpv_embedding(ss_enc,n_stages=n_stages)
        self.lpv_A_Nc = self.lpv_A.map(self.Nc, "thread", 32)
        self.lpv_B_Nc = self.lpv_B.map(self.Nc, "thread", 32)
        self.lpv_C_Nc = self.lpv_C.map(self.Nc, "thread", 32)

    def __call__(self, x_1, dX, u_1, dU):
        list_A = np.zeros([self.Nc*self.nx, self.nx])
        list_B = np.zeros([self.Nc*self.nx, self.nu])
        list_C = np.zeros([self.Nc*self.ny, self.nx])

        return self.lpv_A_Nc(x_1, dX, u_1, dU)
        # return list_A, list_B, list_C

In [4]:
Nc=5

x_1 = np.array([[1,2]]).T
# dX = np.zeros((nx*Nc,1))
dX = np.arange(Nc*nx)[np.newaxis].T*0.01
u_1 = np.array([[3]])
# dU = np.zeros((nu*Nc,1))
dU = np.arange(Nc*nu)[np.newaxis].T*0.01

In [144]:
X_1 = np.zeros((nx,Nc))
X_1[:,0] = x_1[:,0]

for i in range(1,Nc):
    X_1[:,i] = dX[nx*(i-1):nx*i,0] + X_1[:,i-1]

U_1 = np.zeros((nu,Nc))
U_1[:,0] = u_1[:,0]

for i in range(1,Nc):
    U_1[:,i] = dU[nu*(i-1):nu*i,0] + U_1[:,i-1]

In [145]:
X_1, np.hstack(np.split(dX,Nc)), U_1, np.hstack(np.split(dU,Nc))

(array([[1.  , 1.  , 1.02, 1.06, 1.12],
        [2.  , 2.01, 2.04, 2.09, 2.16]]),
 array([[0.  , 0.02, 0.04, 0.06, 0.08],
        [0.01, 0.03, 0.05, 0.07, 0.09]]),
 array([[3.  , 3.  , 3.01, 3.03, 3.06]]),
 array([[0.  , 0.01, 0.02, 0.03, 0.04]]))

In [156]:
embedder = velocity_lpv_embedder(model, Nc)
pA = embedder(X_1, np.hstack(np.split(dX,Nc)), U_1, np.hstack(np.split(dU,Nc)))

In [157]:
list_A = np.zeros([Nc*nx, nx])
for i in range(Nc):
    list_A[(nx*i):(nx*i+nx),:] = pA[:,i*nx:(i+1)*nx]

In [159]:
list_A

array([[ 0.3544466 , -0.15609774],
       [ 0.77745493,  0.56275285],
       [ 0.34787733, -0.15842607],
       [ 0.76551225,  0.55489975],
       [ 0.3304803 , -0.16421011],
       [ 0.73769767,  0.53608424],
       [ 0.3031836 , -0.17325688],
       [ 0.69587011,  0.50732681],
       [ 0.26760427, -0.1851206 ],
       [ 0.64293906,  0.4702993 ]])